In [3]:
# ! mkdir /mnt/data/akshit.jain/visual_reco
# ! pip install openrec
# ! pip install unrar
DATA_DIR='/mnt/data/akshit.jain/visual_reco'
os.chdir(DATA_DIR)

In [10]:
import os
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve

# urlretrieve('http://www.wanghao.in/data/ctrsr_datasets.rar', 'ctrsr_datasets.rar')
os.system('unzip ctrsr_datasets.zip')

0

In [12]:
import numpy as np
import random

users_count = 0
interactions_count = 0
with open('ctrsr_datasets/citeulike-a/users.dat', 'r') as fin:
    for line in fin:
        interactions_count += int(line.split()[0])
        users_count += 1

# radomly hold out an item per user for validation and testing respectively.
val_structured_arr = np.zeros(users_count, dtype=[('user_id', np.int32), ('item_id', np.int32)]) 
test_structured_arr = np.zeros(users_count, dtype=[('user_id', np.int32), ('item_id', np.int32)])
train_structured_arr = np.zeros(interactions_count-11102, dtype=[('user_id', np.int32), ('item_id', np.int32)])

interaction_ind = 0
next_user_id = 0
next_item_id = 0
map_to_item_id = dict()  # Map item id from 0 to len(items)-1

with open('ctrsr_datasets/citeulike-a/users.dat', 'r') as fin:
    for line in fin:
        item_list = line.split()[1:]
        random.shuffle(item_list)
        for ind, item in enumerate(item_list):
            if item not in map_to_item_id:
                map_to_item_id[item] = next_item_id
                next_item_id += 1
            if ind == 0:
                val_structured_arr[next_user_id] = (next_user_id, map_to_item_id[item])
            elif ind == 1:
                test_structured_arr[next_user_id] = (next_user_id, map_to_item_id[item])
            else:
                train_structured_arr[interaction_ind] = (next_user_id, map_to_item_id[item])
                interaction_ind += 1
        next_user_id += 1

In [19]:
from openrec.utils import ImplicitDataset

train_dataset = ImplicitDataset(raw_data=train_structured_arr, 
                        max_user=users_count, 
                        max_item=len(map_to_item_id), name='Train')
val_dataset = ImplicitDataset(raw_data=val_structured_arr, 
                      max_user=users_count,
                      max_item=len(map_to_item_id), name='Val')
test_dataset = ImplicitDataset(raw_data=test_structured_arr, 
                       max_user=users_count,
                       max_item=len(map_to_item_id), name='Test')

In [20]:
from openrec.recommenders import BPR

bpr_model = BPR(batch_size=1000, 
                max_user=train_dataset.max_user(), 
                max_item=train_dataset.max_item(), 
                dim_embed=20, 
                opt='Adam')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [21]:
from openrec.utils.samplers import PairwiseSampler

sampler = PairwiseSampler(batch_size=1000, dataset=train_dataset, num_process=1)

In [22]:
from openrec.utils.evaluators import AUC

auc_evaluator = AUC()

In [23]:
from openrec import ImplicitModelTrainer

model_trainer = ImplicitModelTrainer(batch_size=1000, 
                             test_batch_size=100, 
                            train_dataset=train_dataset, 
                             model=bpr_model, 
                             sampler=sampler)

In [ ]:
model_trainer.train(num_itr=10000, 
                    display_itr=1000, 
                    eval_datasets=[val_dataset, test_dataset],
                    evaluators=[auc_evaluator])

== Start training with FULL evaluation ==
[Itr 100] Finished
[Itr 200] Finished
[Itr 300] Finished
[Itr 400] Finished
[Itr 500] Finished
[Itr 600] Finished
[Itr 700] Finished
[Itr 800] Finished
[Itr 900] Finished
[Itr 1000] Finished
[Itr 1000] loss: 573.331185
..(dataset: Val) evaluation


100%|██████████| 56/56 [00:55<00:00,  1.01it/s]

..(dataset: Val) AUC 0.862679685933231
..(dataset: Test) evaluation



100%|██████████| 56/56 [00:56<00:00,  1.01s/it]


..(dataset: Test) AUC 0.8624182275067398
[Itr 1100] Finished
[Itr 1200] Finished
[Itr 1300] Finished
[Itr 1400] Finished
[Itr 1500] Finished
[Itr 1600] Finished
[Itr 1700] Finished
[Itr 1800] Finished
[Itr 1900] Finished
[Itr 2000] Finished
[Itr 2000] loss: 283.803202
..(dataset: Val) evaluation


...

..(dataset: Test) AUC 0.9476022608079371
[Itr 8100] Finished
[Itr 8200] Finished
[Itr 8300] Finished
[Itr 8400] Finished
[Itr 8500] Finished
[Itr 8600] Finished
[Itr 8700] Finished
[Itr 8800] Finished
[Itr 8900] Finished
[Itr 9000] Finished
[Itr 9000] loss: 57.599665
..(dataset: Val) evaluation


100%|██████████| 56/56 [00:50<00:00,  1.11it/s]

..(dataset: Val) AUC 0.947298609144194
..(dataset: Test) evaluation



100%|██████████| 56/56 [00:51<00:00,  1.09it/s]


..(dataset: Test) AUC 0.9489762742253849
[Itr 9100] Finished
[Itr 9200] Finished
[Itr 9300] Finished
[Itr 9400] Finished
[Itr 9500] Finished
[Itr 9600] Finished
[Itr 9700] Finished


In [39]:
model_trainer._model.save('./bpr-model', 10000)

INFO:tensorflow:./bpr-model/-10000 is not in all_model_checkpoint_paths. Manually adding it.


In [56]:
# ! ls -lh
# model_trainer._model.serve(val_structured_arr)
# print(test_dataset.get_unique_user_list())
affinity=model_trainer._score_full_items(users=val_dataset.get_unique_user_list()[0:1])

In [ ]:
affinity=model_trainer._score_full_items(users=val_dataset.get_unique_user_list()[0:1])

In [58]:
np.max(affinity)

10.891712

In [59]:
import pandas as pd

In [62]:
df=pd.DataFrame(train_structured_arr)

In [66]:
df[df.user_id==234]

,user_id,item_id
8718,234,5605
8719,234,11
8720,234,6548
8721,234,6549
8722,234,1553
8723,234,6550
8724,234,6551
8725,234,5609
8726,234,1875
8727,234,5547


In [82]:
affinity=model_trainer._score_full_items(users=[234x`])

In [91]:
# (-affinity).argsort()[:10]
# idx=
# [k for k,v in map_to_item_id.items() if k==idx]
map_item_to_idx=[(v,k) for k,v in map_to_item_id.items()]

In [94]:
map_item_to_idx=sorted(map_item_to_idx)
map_item_to_idx = [x[1] for x in map_item_to_idx]
map_item_to_idx=np.array(map_item_to_idx)

In [ ]:
idf[idf['doc.id'].isin(map_item_to_idxdf[df.user_id==234].item_id.tolist())].title

In [70]:
idf=pd.read_csv('ctrsr_datasets/citeulike-a/raw-data.csv', encoding='ISO-8859-1')

In [109]:
idf[idf['doc.id'].isin(map_item_to_idx[df[df.user_id==15].item_id.tolist()])].title

822      framing processes and social movements an over...
4178                       prokaryotes the unseen majority
6358            towards multidimensional genome annotation
12825    a biomolecular force field based on the free e...
15707    the gene ontologys reference genome project a ...
16044    finding structure with randomness stochastic a...
16097    how to identify essential genes from molecular...
16318    a quick guide for developing effective bioinfo...
Name: title, dtype: object

In [ ]:
model_trainer._model.embed

In [102]:
map_item_to_idx[df[df.user_id==234].item_id.tolist()]

array(['5606', '3332', '512', '11036', '6432', '9678', '5722', '6528',
       '6715', '4557', '12015', '2642'], dtype='<U5')